In [1]:
%load_ext autoreload
%autoreload 2
    
import tensorflow as tf
from tensorflow  import keras
import numpy as np

import librosa 
from dataset import WavDataset
import matplotlib.pyplot as plt
from tqdm import tqdm
from config import *
import h5py
import soundfile as sf
from scipy.signal import resample_poly
from util import DEFAULT_TOKENS
import librosa

2024-09-29 18:02:47.085080: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 18:02:47.085132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 18:02:47.235553: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 18:02:47.530446: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 18:02:49.179088: W tensorflow/comp

In [2]:
from models import build_resnet16
from keras import metrics

X_shape = (40, 107, 1)

model = build_resnet16(input_shape=X_shape)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[
                metrics.Recall(thresholds=0.5),
                metrics.Precision(thresholds=0.5),
                metrics.AUC(curve='pr', name='auc_pr')])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 40, 107,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 40, 107,   │        160 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 40, 107,   │         64 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 40, 107,   │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 20, 54,    │      2,320 │ re_lu[0][0]       │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 54,    │         64 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 20, 54,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 20, 54,    │      2,320 │ re_lu_1[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 20, 54,    │        272 │ re_lu[0][0]       │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 54,    │         64 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 54,    │         64 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 20, 54,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 20, 54,    │          0 │ add[0][0]         │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 20, 54,    │      2,320 │ re_lu_2[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 54,    │         64 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 20, 54,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 20, 54,    │      2,320 │ re_lu_3[0][0]   

 Total params: 176,596 (689.83 KB)

 Trainable params: 175,444 (685.33 KB)

 Non-trainable params: 1,152 (4.50 KB)

In [3]:
from preprocessing import split

X = np.load(INTERMEDIATE / 'logmel.npy')
Y = np.load(INTERMEDIATE / 'logmel_labels.npy')
train, pool, test = split(X, Y)
X_train, Y_train = train
X_pool, Y_pool = pool
X_test, Y_test = test
(X_pool.shape, Y_pool.shape), (X_train.shape, Y_train.shape), (X_test.shape, Y_test.shape)

(((135462, 40, 107), (135462, 4)),
 ((33866, 40, 107), (33866, 4)),
 ((42332, 40, 107), (42332, 4)))

In [ ]:
from preprocessing import undersample, oversample_minority_classes

X_train_full = np.concatenate([X_train, X_pool], axis=0)
Y_train_full = np.concatenate([Y_train, Y_pool], axis=0)

histories = []
for epoch in range(10):
    print(f'-- epoch {epoch+1}/10 ---')
    X_resampled, Y_resampled = undersample(X_train_full, Y_train_full, reduce_to=0.9)
    X_resampled, Y_resampled = oversample_minority_classes(X_resampled, Y_resampled)
    hist =  model.fit(
        x=X_resampled, y=Y_resampled, 
        batch_size=32,
        validation_data=(X_test, Y_test),
        epochs=1,
        verbose=2
    )
    histories.append(hist.history)

histories

-- epoch 1/10 ---


2024-09-29 18:03:29.533904: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2629392320 exceeds 10% of free system memory.


In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve

def plot_pr(name, labels, predictions, **kwargs):
    prec, rec, _ = precision_recall_curve(labels, predictions)
    plt.plot(prec, rec, label=name, linewidth=2, **kwargs)
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.grid(True)


def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = roc_curve(labels, predictions)
    plt.plot(fp*100, tp*100, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.xlim([0,100])
    plt.ylim([0,100])
    plt.grid(True)

Y_pred = model.predict(X_test, verbose=2)

labels = DEFAULT_TOKENS.keys()
for i, label in enumerate(labels):
    plot_roc(labels, Y_test[:, i], Y_pred[:, i])
    
plt.legend(labels) 
plt.show() 

for i, label in enumerate(labels):
    plot_pr(labels, Y_test[:, i], Y_pred[:, i])
    
plt.legend(labels) 
plt.show() 

In [ ]:
Y_pool.sum(axis=0)

In [ ]:
from util import LABELS

for i, n in enumerate([688, 741, 274, 444]):
    print(f'{LABELS[i]} -> {n}')